Tournament keys -- had to lookup online @ https://coverage-matrix.sportradar.com/

Wimbledon 
- MS: sr:competition:2555
- WS: sr:competition:2559


French
- MS: sr:competition:2579
- WS: sr:competition:2583


Australian
- MS: sr:competition:2567
- WS: sr:competition:2571


US
- MS: sr:competition:2591	
- WS: sr:competition:2595	

In [81]:
# Import packages
import http.client
import json
import pandas as pd
from datetime import datetime
import numpy as np

# Api keys
eda_api_key = "thp5rpgrbegb8hkc7zrapy36"
mds_api_key = "fqe4f9fyp8m84mg679xu7dnj"

# Variable to count api calls
global CALLS
CALLS = 0

# Make an API call given get, return a json object
def send_req(get):
    global CALLS
    # Connect to SR API
    conn = http.client.HTTPSConnection("api.sportradar.com")
    conn.request("GET", get)
    with conn.getresponse() as res:
        # Check if the response status is OK (200)
        if res.status == 200:
            data = res.read().decode("utf-8")
            
            # Check if data is not empty before decoding
            if data:
                CALLS += 1
                return json.loads(data)
            else:
                print("Error: Empty response data.")
                return None
        else:
            print(f"Error: Failed request with status code {res.status}")
            return None

# Display json object neatly
def pp(json_obj):
    print(json.dumps(json_obj, indent=4))

# Grandslam competition codes for API
M_GRAND_SLAMS = ["sr:competition:2555","sr:competition:2579","sr:competition:2567", "sr:competition:2591"]
W_GRAND_SLAMS = ["sr:competition:2559","sr:competition:2583","sr:competition:2571","sr:competition:2595"]

# temp competition variable (Wimbledon)
competition = M_GRAND_SLAMS[0]

In [78]:
# Get json object of the seasons (returns the top 3) related to the competition (tourney)
def get_competition_seasons(comp):
    json = send_req("/tennis/trial/v3/en/competitions/"+comp+"/seasons.json?api_key="+eda_api_key)
    if json != None:
        return json['seasons']
    else: return None

seasons_json = get_competition_seasons(competition)

In [27]:
# Get the season ids and add to a list
# note that 2024 seasons have not happened yet so they're empty
# Theyre ordered chronologically
def return_seasons(seasons_json):
    # Create a dictionary of season ids and their years 
    seasons = {}
    for sj in seasons_json:
        seasons[sj['id']] = sj['year']
    return seasons

# set season to the 
seasons = return_seasons(seasons_json)

# temporary variable 
sea = list(seasons.keys())[1] # apparently 2022 doesn't have play by plays?

In [62]:
# Gets information about every match in the season (tournament)
def get_summary(sea):
    print(sea)
    get = "/tennis/trial/v3/en/seasons/"+sea+"/summaries.json?api_key="+eda_api_key+"&start=200"
    return send_req(get)

#summary = get_summary(sea)

In [68]:
# Seeing which matches after the qualifiers have play by play stats
def list_of_pbps(summary):
    pbps = []
    if summary == None:
        return pbps
    for match in summary["summaries"]:
        round = match['sport_event']['sport_event_context']['round']['name']
        if round in ['round_of_64','round_of_32','round_of_16','semifinal','final',]: 
            if match["sport_event"]['coverage']['sport_event_properties']['play_by_play']==True:
                pbps.append(match["sport_event"]["id"])
    return pbps

# pbps = list_of_pbps(summary)
# print(pbps)
# print(len(pbps))

In [79]:

matches = []

for comp in M_GRAND_SLAMS[0:2]:
    seasons_json = get_competition_seasons(comp)
    seasons = return_seasons(seasons_json)
    for sea in list(seasons.keys()):
        summary = get_summary(sea)
        pbps = list_of_pbps(summary)
        if not pbps
        matches.append(pbps)

print(CALLS)
pp(matches)
    

sr:season:77033
{'generated_at': '2024-01-24T18:04:05+00:00', 'summaries': [{'sport_event': {'id': 'sr:sport_event:34242977', 'start_time': '2022-06-30T11:40:00+00:00', 'start_time_confirmed': True, 'sport_event_context': {'sport': {'id': 'sr:sport:5', 'name': 'Tennis'}, 'category': {'id': 'sr:category:3', 'name': 'ATP'}, 'competition': {'id': 'sr:competition:2555', 'name': 'Wimbledon Men Singles', 'parent_id': 'sr:competition:2553', 'type': 'singles', 'gender': 'men', 'level': 'grand_slam'}, 'season': {'id': 'sr:season:77033', 'name': 'Wimbledon Men Singles 2022', 'start_date': '2022-06-20', 'end_date': '2022-07-10', 'year': '2022', 'competition_id': 'sr:competition:2555'}, 'stage': {'order': 2, 'type': 'cup', 'phase': 'stage_1_playoff', 'start_date': '2022-06-27', 'end_date': '2022-07-10', 'year': '2022'}, 'round': {'name': 'round_of_64'}, 'groups': [{'id': 'sr:cup:126949', 'name': '2022 Wimbledon, London, Great Britain'}], 'mode': {'best_of': 5}}, 'coverage': {'type': 'sport_event',

In [33]:
# Query sport event timeline
def rq_sport_event_timeline(event, api_key):
    pre = "/tennis/trial/v3/en/sport_events/"
    post = "/timeline.json"
    get = pre+event+post+"?api_key="+api_key
    return send_req(get)

#timeline = rq_sport_event_timeline("sr:sport_event:46749693",eda_api_key)

timeline0 = rq_sport_event_timeline(pbps[0], eda_api_key)
timeline1 = rq_sport_event_timeline(pbps[1], eda_api_key)
pp(timeline0)


{
    "generated_at": "2024-01-24T17:53:02+00:00",
    "sport_event": {
        "id": "sr:sport_event:42001123",
        "start_time": "2023-07-06T17:00:00+00:00",
        "start_time_confirmed": true,
        "sport_event_context": {
            "sport": {
                "id": "sr:sport:5",
                "name": "Tennis"
            },
            "category": {
                "id": "sr:category:3",
                "name": "ATP"
            },
            "competition": {
                "id": "sr:competition:2555",
                "name": "Wimbledon Men Singles",
                "parent_id": "sr:competition:2553",
                "type": "singles",
                "gender": "men",
                "level": "grand_slam"
            },
            "season": {
                "id": "sr:season:95307",
                "name": "Wimbledon Men Singles 2023",
                "start_date": "2023-06-26",
                "end_date": "2023-07-16",
                "year": "2023",
               

In [247]:
def str_to_time(str):
    return datetime.fromisoformat(str)


def play_by_play(timeline):

    # avg point length??
    # avg points per game
    # avg (or each) game length
    # avg (or each) set length

    # Is it a match point?
     

    # for each event in the timeline
    set_time, game_time = None
    set_lengths, game_lengths = []
    points_per_game = []
    in_game = False

    suspended = False
    for time in timeline['timeline']:
        if time['type']!='point':
            
            # Compute set length
            if time['type'] in ['period_start','match_ended'] and suspended==False: # beginning/ends of sets
                if "period_name" in time and time['period_name']=='suspended':
                    suspended = True
                    continue # Neglecting any part of the game that follows a suspension (I think this can be fixed!!!!! but keep for now)

                curr_time = str_to_time(time['time'])
                if set_time != None:
                    set_lengths.append( (curr_time-set_time).total_seconds() /60) # subtracting two set times to get set length in minutes
                set_time = curr_time # start of next step time

            # Compute game length
            if time['type'] in ['period_score','period_start'] and suspended ==False: # start of a set or game
                curr_time = str_to_time(time['time']) # TODO: PRINT THESE OUT TO CHECK, WHY IS THERE A ZERO MINUTE ONE, CORRECT NUMBER?
                if game_time!=None:
                    game_lengths.append((curr_time-game_time).total_seconds()/60)
                game_time = curr_time # start of next step time

            # Compute match length

            continue
        else:
    

    print(set_lengths)
    print(game_lengths)



In [248]:
play_by_play(timeline)

home:6
away:10
[40.56666666666667, 44.81666666666667, 44.81666666666667]
[3.25, 2.0833333333333335, 6.15, 4.166666666666667, 3.3833333333333333, 13.183333333333334, 2.033333333333333, 3.9833333333333334, 2.3333333333333335, 0.0, 4.533333333333333, 4.016666666666667, 3.5, 5.0, 4.7, 5.116666666666666, 4.266666666666667, 3.15, 6.566666666666666, 3.966666666666667, 0.0, 6.85, 2.05, 1.9833333333333334, 2.466666666666667, 2.9166666666666665, 6.066666666666666, 2.966666666666667, 4.616666666666666, 5.033333333333333, 9.85]


In [170]:
def get_event_stats(timeline):

    event_id = timeline['sport_event']['id']
    stats = [pd.DataFrame(),pd.DataFrame()]
    context = timeline['sport_event']['sport_event_context']

    for i in range(len(stats)):
        comp_info = timeline['statistics']['totals']['competitors'][i]
        stats_i = pd.json_normalize(comp_info['statistics'])
        stats_i.insert(0, 'event_id', event_id)
        stats_i.insert(1, 'player_id', comp_info['id'])
        stats_i.insert(2, 'name', comp_info['name'])
        try:
            seed = timeline['sport_event']['competitors'][i]['seed']
        except:
            seed = None
        stats_i.insert(3, 'seed', seed)
        # MAKE SURE 1 AND 2 MATCH HOME AND AWAY
        
        stats[i] = stats_i

    merged = pd.merge(stats[0], stats[1], how='outer', left_on = 'event_id', right_on='event_id', suffixes=("_home", "_away"))

    # Men or womens match
    merged['men'] = (context['competition']['gender']=="men")

    # Which round?
    merged['round'] = context['round']['name']

    # How many sets
    merged['best_of'] = context['mode']['best_of']

    # Who won?
    merged['win_1'] = timeline['sport_event_status']['home_score']>timeline['sport_event_status']['away_score']

    # Getting scores
    i = 1
    for set_ in timeline['sport_event_status']['period_scores']:
        merged['set'+str(i)+"_diff"] = set_['home_score']-set_['away_score']
        merged['set'+str(i)+'_games'] = set_['home_score']+set_['away_score']
        i+=1

    # Get play by play

    return merged

stats0 = get_event_stats(timeline0)
stats1 = get_event_stats(timeline1)
stats_df = pd.concat([stats0, stats1]).reset_index()
stats_df

,index,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,...,men,round,best_of,win_1,set1_diff,set1_games,set2_diff,set2_games,set3_diff,set3_games
0,0,sr:sport_event:42001123,sr:competitor:89320,"Ivashka, Ilya",None,5,15,18,2,2,...,True,round_of_64,5,False,-3,9,-2,10,-2,10
1,0,sr:sport_event:42001113,sr:competitor:163504,"Medvedev, Daniil",3,7,22,13,15,5,...,True,round_of_64,5,True,3,9,3,9,1,13


In [140]:
stats_df.to_csv("data/stats_df.csv", index=False)

In [122]:
stats_df = pd.read_csv("data/stats_df.csv")

In [ ]:
stats_df

In [24]:
# get competitors' stats for an individual game
all_stats = pd.DataFrame()
for comp_stats in timeline['statistics']['totals']['competitors']:
    stats = pd.json_normalize(comp_stats['statistics'])
    stats.insert(0, 'player_id', comp_stats['id'])
    stats.insert(1, 'name', comp_stats['name'])

    print(stats)

                     id             name  aces  breakpoints_won  \
0  sr:competitor:387066  Forejtek, Jonas     4                4   

   double_faults  first_serve_points_won  first_serve_successful  games_won  \
0              1                      24                      28         12   

   max_games_in_a_row  max_points_in_a_row  points_won  \
0                   4                    6          61   

   second_serve_points_won  second_serve_successful  service_games_won  \
0                       12                       25                  8   

   service_points_lost  service_points_won  tiebreaks_won  total_breakpoints  
0                   18                  36              0                  9  
                     id              name  aces  breakpoints_won  \
0  sr:competitor:283759  Chidekh, Clement     2                1   

   double_faults  first_serve_points_won  first_serve_successful  games_won  \
0              7                      21                      32  

Unused code

In [ ]:

# Break points
# Is it a break point?
# if is_break_point(int(time['home_score']), int(time['away_score']), time['server']):
#    if int(time['home_score'])>int(time['away_score']):
#       break_point_home+=1
#    else:
#       break_point_away+=1

# def is_break_point(home_score, away_score, server):

#     if home_score==away_score: # score is tied
#         return False
#     elif (home_score>away_score and server=='home') or (away_score>home_score and server=='away'): # server is winning
#         return False
#     elif not (home_score==40 or away_score==40): # not at the end of the game
#         return False
#     else:
#         return True